In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

notebook_path = Path().absolute()
sys.path.append(str(notebook_path.parent))

In [3]:
import torch
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from neural_controllers import NeuralController

In [4]:
from utils import shakespeare_dataset

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [5]:
model_type = 'llama'

if model_type=='llama':

    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

    language_model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="cuda"
    )

    use_fast_tokenizer = "LlamaForCausalLM" not in language_model.config.architectures
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
    tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
    model_name='llama_3_8b_it'
    assistant_tag = '<|start_header_id|>assistant<|end_header_id|>'
    
elif model_type=='gemma':

    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
    language_model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2-9b-it",
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    model_name='gemma_2_9b_it'


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
controller = NeuralController(
        language_model,
        tokenizer
    )

n_components: 5
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 10
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5
calibrate            : False



In [7]:
concept_types = ['english', 'shakespeare']

data_dir = "../data/languages"

data = shakespeare_dataset(data_dir, concept_types, controller, assistant_tag)

train 200 test 200
train 200 test 200


In [8]:
controllers = {}
for concept_type in tqdm(concept_types):
    
    other_type = [k for k in concept_types if k != concept_type][0]
    
    train_data = data[concept_type]['train']
    test_data = data[concept_type]['test']
        
    language_controller = NeuralController(
        language_model,
        tokenizer,
        rfm_iters=8,
        batch_size=2,
        n_components=5,
        control_method='rfm'
    )
    
    language_controller.compute_directions(train_data['inputs'], train_data['labels'])
    
    controllers[concept_type] = language_controller
    

  0%|          | 0/2 [00:00<?, ?it/s]

n_components: 5
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5
calibrate            : False

Tuning metric: auc
Getting activations from forward passes



100%|██████████| 80/80 [00:07<00:00, 10.14it/s]


Getting activations from forward passes



100%|██████████| 20/20 [00:01<00:00, 10.30it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...


 97%|█████████▋| 30/31 [00:18<00:01,  1.03s/it]

Optimal M batch size: 160
Time taken for round 7: 0.00455164909362793 seconds
Optimal M batch size: 160
Best RFM auc: 1.0, reg: 0.001, bw: 1, center_grads: True
Time taken to train rfm probe: 0.20527362823486328 seconds
Time taken to compute eigenvectors: 0.010685443878173828 seconds
train X shape: torch.Size([160, 4096]) train y shape: torch.Size([160, 1]) val X shape: torch.Size([40, 4096]) val y shape: torch.Size([40, 1])
Fitting RFM with ntrain: 160, d: 4096, and nval: 40
Optimal M batch size: 160
Time taken for round 0: 0.003370523452758789 seconds
Optimal M batch size: 160
Time taken for round 1: 0.0034034252166748047 seconds
Optimal M batch size: 160
Time taken for round 2: 0.004860877990722656 seconds
Optimal M batch size: 160
Time taken for round 3: 0.004319429397583008 seconds
Optimal M batch size: 160
Time taken for round 4: 0.003410816192626953 seconds
Optimal M batch size: 160
Time taken for round 5: 0.0033121109008789062 seconds
Optimal M batch size: 160
Time taken for ro


100%|██████████| 31/31 [00:18<00:00,  1.70it/s]


Optimal M batch size: 160
Time taken for round 2: 0.004784345626831055 seconds
Optimal M batch size: 160
Time taken for round 3: 0.0038933753967285156 seconds
Optimal M batch size: 160
Time taken for round 4: 0.0032815933227539062 seconds
Optimal M batch size: 160
Time taken for round 5: 0.003480672836303711 seconds
Optimal M batch size: 160
Time taken for round 6: 0.0034744739532470703 seconds
Optimal M batch size: 160
Time taken for round 7: 0.003577709197998047 seconds
Optimal M batch size: 160
Best RFM auc: 1.0, reg: 0.001, bw: 1, center_grads: True
Time taken to train rfm probe: 0.20703125 seconds
Time taken to compute eigenvectors: 0.014101505279541016 seconds



100%|██████████| 2/2 [00:49<00:00, 24.89s/it]


In [9]:
for concept_type in concept_types:
    controller = controllers[concept_type]
    other_type = [k for k in concept_types if k!=concept_type][0]
    
    controller.save(concept=f'{concept_type}_{other_type}', model_name=model_name, path='../directions/')

# Control

In [10]:
concept_types = ['english', 'shakespeare']
# concept_types = ['english', 'german']

controllers = {}

for concept_type in concept_types:
    
    controller = NeuralController(
        language_model,
        tokenizer,
        rfm_iters=8,
        control_method='rfm'
    )
    
    other_type = [k for k in concept_types if k!=concept_type][0]
    
    controller.load(concept=f'{concept_type}_{other_type}', model_name=model_name, path='../directions/')
    
    controllers[concept_type] = controller

n_components: 5
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5
calibrate            : False

Detector found
n_components: 5
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 2
M_batch_size         : 2048
n_components         : 5
calibrate            : False

Detector found


In [14]:
concept_type = "english"
language_controller = controllers[concept_type]

raw_inputs = [
    # "How are you today?",
    # "What can I buy in a grocery store?",
    f"What can I do to treat flu symptoms?",
]
inputs = [language_controller.format_prompt(x) for x in raw_inputs]
num_new_tokens = 150


#llama 
coef=0.6
layers = list(range(-1, -31, -1))

#gemma
# coef=9
# layers = list(range(-1, -41, -1))
assistant_tag = '<|start_header_id|>assistant<|end_header_id|>'
gens=[]
print()
for i in inputs:
    print("Prompt:", i)
    start_idx = len(i)
        
    print("===== No Control =====")
    gen1 = language_controller.generate(i, max_new_tokens=num_new_tokens, do_sample=False)[start_idx:]
    print(gen1)
    print()
    
    print(f"===== + {concept_type} Control =====")
    gen2 = language_controller.generate(i, layers_to_control=layers, control_coef=coef, 
                                max_new_tokens=num_new_tokens, do_sample=False)[start_idx:].replace(assistant_tag,'')
    print(gen2)
    print()
    print()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Prompt: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What can I do to treat flu symptoms?<|eot_id|>
===== No Control =====


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

Treating flu symptoms involves a combination of self-care, over-the-counter medications, and home remedies. Here are some effective ways to alleviate flu symptoms:

**Self-Care:**

1. **Stay hydrated**: Drink plenty of fluids, such as water, clear broths, and electrolyte-rich beverages like sports drinks.
2. **Rest**: Get plenty of sleep and rest to help your body recover.
3. **Use a humidifier**: Add moisture to the air to relieve congestion and cough.
4. **Practice good hygiene**: Wash your hands frequently, avoid close contact with others, and avoid sharing utensils or personal items.

**Over-the-Counter Medications:**

1. **Pain relievers**: Use acet

===== + english Control =====


Good sir or madam, thou seekest relief from the woes of the flu? Verily, I shall impart upon thee some tried and true remedies to alleviate the symptoms of this most foul affliction.

Firstly, 'tis essential to rest and conserve thy energies, for the flu do